In [ ]:
file_names
all_results
file_order

# Calculate the average and coefficient of variation for each file
averaged_results = {file: np.mean(all_results[file], axis=0) for file in file_names}
averaged_training_data = {file: result[0] for file, result in averaged_results.items()}
averaged_all_data = {file: result[1] for file, result in averaged_results.items()}
CV_training_data = {file: np.std([result[0] for result in all_results[file]]) / np.mean([result[0] for result in all_results[file]]) for file in file_names}
CV_all_data = {file: np.std([result[1] for result in all_results[file]]) / np.mean([result[1] for result in all_results[file]]) for file in file_names}

# Determine sorted orders for training and all data
training_data_order = sorted(file_names, key=lambda file: averaged_training_data[file])
all_data_order = sorted(file_names, key=lambda file: averaged_all_data[file])

# Ensure file order is respected
file_order = [file for file in file_order if file in file_names]
x = np.arange(len(file_order))
bar_width = 0.35

# Extracting the individual and average results
training_data_mse = {file: [result[0] for result in all_results[file]] for file in file_order}
all_data_mse = {file: [result[1] for result in all_results[file]] for file in file_order}
avg_training_data = [averaged_training_data[file] for file in file_order]
avg_all_data = [averaged_all_data[file] for file in file_order]

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 4))
for i, file in enumerate(file_order):
	ax1.scatter([x[i]] * len(training_data_mse[file]), training_data_mse[file], color='skyblue', label='Training Data MSE' if i == 0 else "")
	ax1.hlines(avg_training_data[i], x[i] - 0.1, x[i] + 0.1, colors='grey', linestyles='solid', label='Average MSE' if i == 0 else "")
ax1.set_ylim(0, max(max(max(training_data_mse.values())), max(avg_training_data)) * 1.2)
for i, file in enumerate(file_order):
	ax1.text(x[i], max(training_data_mse[file]) * 1.05, f'{int(CV_training_data[file]*100)}%', ha='center', va='bottom', color='black', fontsize=8)
ax1.set_xticks(x, file_order, rotation=45, ha='right', rotation_mode='anchor')
ax1.set_ylabel('Mean Squared Error (MSE)')
ax1.set_title('Testing against self')

for i, file in enumerate(file_order):
	ax2.scatter([x[i]] * len(all_data_mse[file]), all_data_mse[file], color='lightgreen', label='All Data MSE' if i == 0 else "")
	ax2.hlines(avg_all_data[i], x[i] - 0.1, x[i] + 0.1, colors='grey', linestyles='solid', label='Average MSE' if i == 0 else "")
ax2.set_ylim(0, max(max(max(all_data_mse.values())), max(avg_all_data)) * 1.2)
for i, file in enumerate(file_order):
	ax2.text(x[i], max(all_data_mse[file]) * 1.05, f'{int(CV_all_data[file]*100)}%', ha='center', va='bottom', color='black', fontsize=8)
ax2.set_xticks(x, file_order, rotation=45, ha='right', rotation_mode='anchor')
ax2.set_title('Testing against aggregation')
##plt.savefig('Images/all_data_mse.png')
##plt.savefig('Images/Figure_All_Data.pdf', format='pdf')
fig.supxlabel("Dataset")
fig.tight_layout()
fig.show()